In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/TSForecasting/Data/consolidatedDataForPackage.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [3]:
def fullDay(data):
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

def hourlySamples(df):
    groupkey = df['Display Time'].values.astype('datetime64[h]')
    result = df.groupby(groupkey).mean()
    result = result.reset_index(drop=True)
    return (result)

def subSample(data):
    data = data.reset_index(drop=True)
    data['Display Time'] = pd.to_datetime(data['Display Time'])
    data['time_gap'] = data['Display Time'].shift(1)-data['Display Time'][0]
    data['time_gap'][0] = '00:00:00'
    mods = [0,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899]
    subset = pd.DataFrame()
    for i in range(1,len(data.index)):
        seconds = data['time_gap'][i].total_seconds()
        if (seconds%900) in mods:
            subj_id = data['subjectId'][i]
            gv = data['GlucoseValue'][i]
            dt = data['Display Time'][i]
            temp_df = pd.DataFrame({'Display Time':[dt], 'GlucoseValue':[gv], 'subjectId':[subj_id]})
            subset = pd.concat([temp_df,subset],ignore_index=True)
    subset = subset.iloc[::-1]
    subset = subset.reset_index(drop=True)
    data.drop(['time_gap'], axis=1, inplace=True)
    return subset

In [4]:
def IGC(df, unit, lltr = 80, ultr = 140, a = 1.1, b = 2.0, c = 30, d = 30):
    if unit == 'mg':
        gv = df['GlucoseValue']
    elif unit == 'mmol':
        gv = 18*df['GlucoseValue']
    else:
        print('Unit should either be mg or mmol')
        return 0
    
    lower_gv = gv[gv < 90]
    upper_gv = gv[gv > 140]
    
    
    count_lower = len(lower_gv.index)
    count_upper = len(upper_gv.index)
    
    hypoglycemicIndex = np.sum(np.power((lltr - lower_gv), b)) / (count_lower*d)   
    hyperglycemicIndex = np.sum(np.power((upper_gv - ultr), a)) / (count_upper*c)
    
    if np.isnan(hypoglycemicIndex):
        hypoglycemicIndex = 0
    if np.isnan(hyperglycemicIndex):
        hyperglycemicIndex=0
    
    igc = hypoglycemicIndex + hyperglycemicIndex
    return igc, hypoglycemicIndex, hyperglycemicIndex

In [8]:
def glucoseLiabilityIndex(data, unit):
    data = hourlySamples(data)
    if unit == 'mg':
        data['GlucoseValue'] = data['GlucoseValue']/18
    gli = np.sum(np.power(data['GlucoseValue'][i] - data['GlucoseValue'][i+1],2) for i in range(0, len(data.index)-1))
    return round(gli)

In [6]:
def adrr(xx, unit):
    
    if unit == 'mg':
        f_bg = 1.509*(np.log(xx['GlucoseValue'])**1.084)-5.381
        xx['F(BG)'] = f_bg
    elif unit == 'mmol':
        f_bg = 1.509*(np.log(xx['GlucoseValue']*18)**1.084)-5.381
        xx['F(BG)'] = f_bg
    else:
        print('Unit should either be mg or mmol')
        return 0
    
    dates = []
    for i in range(len(xx.index)):
        dates.append(xx['Display Time'][i].date())
    xx['Date'] = dates 
    

    for Date, df in xx.groupby('Date'):
        r_BG = 0
        rl_BG = [0]
        rh_BG = [0]
        LR = 0
        HR = 0
        ADDR_daily = []
        for f_BG in df['F(BG)']:
            if f_BG < 0:
                rl_BG.append(f_BG)
            else:
                rh_BG.append(f_BG)

        LR = max(rl_BG)
        HR = max(rh_BG)
        ADDR_daily.append(LR+HR)
    
    
    return round(mean(ADDR_daily),3)
        
            

In [16]:
def modd(data):
    data = subSample(data)
    data['Display Time'] = data['Display Time'].dt.round('5min') 
    
    times = []
    for i in range(len(data.index)):
        times.append(data['Display Time'][i].time())
    data['Time'] = times  


    Modd = [] 
    s = 0
    gvDiff = 0

    for Time, df in data.groupby('Time'):
        gvDiff = df['GlucoseValue'] - df['GlucoseValue'].shift(-1)
        s = round(gvDiff.sum(),3)
        Modd.append(s)
    return round(mean(Modd),3)
    

In [22]:
xx = data[data['subjectId'] == "ID01"]
xx = fullDay(xx)

In [23]:
IGC(xx, 'mg')

(4.566541360725955, 2.334945176222724, 2.2315961845032306)

In [24]:
glucoseLiabilityIndex(xx, 'mg')

321.0

In [25]:
adrr(xx, 'mg')

3.871

In [26]:
modd(xx)

3.526